## Initialize

In [ ]:


import requests
import pandas as pd
import os
import sys
from datetime import datetime
import base64

sys.path.append(os.path.join(sys.path[0], '..', '..', '..', '..'))

import src.esh_client as esh
from demo.shared.tools import TENANT_SUFFIX, get_base_url, get_root_path
base_url = get_base_url()
root_path = get_root_path()
example_name = sys.path[0].split(os.sep)[-2]


def execute_search_query(query):
    r = requests.post(
        f'{base_url}/v1/query/{example_name}{TENANT_SUFFIX}/latest', json=[query])
    if r.status_code == 200:
        res = r.json()[0]
        if '@odata.count' in res:
            print(f'Total count: {res["@odata.count"]}')
        if res and 'value' in res and res['value']:
            df = pd.DataFrame.from_dict(res['value'])[
                ['title', 'author', 'text', 'createdAt', 'changedAt']]
            df.rename(columns={'title': 'Title', 'author': 'Author',
                      'text': 'Text', 'createdAt': 'Created at', 'changedAt': 'Changed at'}, inplace=True)
            display(df)
        else:
            print('No results')
    else:
        print(f'Error during search. HTTP status code {r.status_code}')
        print(r.json())


## Text search

In [ ]:
so = esh.EshObject(
    count=True,
    top=3,
    wherefound=True,
    scope=['Document'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property=['text']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='BMW')
            )
        ]
    )
)
execute_search_query(so.dict(exclude_none=True))

## Search for several text snippets

In [ ]:
so = esh.EshObject(
    count=True,
    top=4,
    scope=['Document'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property=['text']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='BMW Finance')
            ),
            
            esh.Comparison(
                property=esh.Property(property=['text']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Finance'))
        ]
    )
)
execute_search_query(so.dict(exclude_none=True))

## Search for explicit numbers

In [ ]:
so = esh.EshObject(
    count=True,
    top=4,
    scope=['Document'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property=['text']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='2,340', isPhrase=True)
            )]
    )
)
execute_search_query(so.dict(exclude_none=True))

## Search for author

In [ ]:
so = esh.EshObject(
    count=True,
    top=4,
    scope=['Document'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property=['author']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Daria')
            )]
    )
)
execute_search_query(so.dict(exclude_none=True))

## Search for date when the documents were created

In [ ]:
so = esh.EshObject(
    count=True,
    top=4,
    scope=['Document'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property=['createdAt']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='2022-10-27')
            )]
    )
)
execute_search_query(so.dict(exclude_none=True))

## Fuzzy Search

In [ ]:
so = esh.EshObject(
    count=True,
    # top=1,
    scope=['Document'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property=['text']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(
                    value='stretgy', # strategy
                    searchOptions=esh.SearchOptions(fuzzinessThreshold=0.7)
                )
            )
        ]
    )
)

execute_search_query(so.dict(exclude_none=True))

## Search for creation date ranges

In [ ]:
so = esh.EshObject(
    count=True,
    top=4,
    scope=['Document'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property=['createdAt']),
                operator=esh.ComparisonOperator.BetweenCaseInsensitive,
                value=esh.RangeValue(start=esh.DateValue(value='2022-10-01'), end=esh.DateValue(value='2022-10-31'))
            )]
    )
)
execute_search_query(so.dict(exclude_none=True))

## Search for explizit text in a file

In [ ]:
so = esh.EshObject(
    count=True,
    top=4,
    scope=['Document'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property=['text']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='BMW Group und Daimler Mobility veräußern PARK NOW an EasyPark Group', isPhrase=True)
            )]
    )
)
execute_search_query(so.dict(exclude_none=True))